In [ ]:
import dautil as dl
import ch7util
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from IPython.display import HTML

In [ ]:
context = dl.nb.Context('autoregressive_test')
lr = dl.nb.LatexRenderer(chapter=7, start=13, context=context)
lr.render(r'X_t = c + \sum_{i=1}^p \varphi_i X_{t-i}+ \varepsilon_t')
lr.render(r'X_t = c + \varepsilon_t')
lr.render(r'Market Effciency = 1 -  \frac{  R_{Restricted} ^{2} }{R_{Unrestricted} ^{2}}')

In [ ]:
ohlc = dl.data.OHLC()
efficiencies = []
restricted_r2 = []
unrestricted_r2 = []

for stock in ch7util.STOCKS:
    rets = ch7util.log_rets(ohlc.get(stock)['Adj Close'])
    restricted = sm.OLS(rets, rets.mean() * np.ones_like(rets)).fit()
    rets_1 = rets[3:-1]
    rets_2 = rets[2:-2]
    rets_3 = rets[1:-3]
    rets_4 = rets[:-4]
    x = np.vstack((rets_1, rets_2, rets_3, rets_4)).T
    x = sm.add_constant(x)
    y = rets[4:]
    unrestricted = sm.OLS(y, x).fit()
    restricted_r2.append(restricted.rsquared)
    unrestricted_r2.append(unrestricted.rsquared)
    efficiencies.append(1 - restricted.rsquared/unrestricted.rsquared)

In [ ]:
%matplotlib inline
dl.options.mimic_seaborn()
dl.nb.RcWidget(context)
dl.nb.LabelWidget(2, 1, context)

In [ ]:
sp = dl.plotting.Subplotter(2, 1, context)
dl.plotting.bar(sp.ax, ch7util.STOCKS, efficiencies)
sp.label()
dl.plotting.plot_text(sp.next_ax(), unrestricted_r2, np.array(restricted_r2)/10 ** -16,
                      ch7util.STOCKS, add_scatter=True)
sp.label()
HTML(sp.exit())